Notebook responsável para realizar transformações no dataframe de contratos. Ele está sendo usado apenas para testes, código completos serão enviados para o arquivo de transform.py

Carregamento do arquivo bruto de contratos 👇

In [2]:
from pathlib import Path
import pandas as pd

# Configura o caminho para os arquivos brutos
raw_data_dir = Path('..') / 'data' / 'raw'

# Localiza o arquivo de contratos mais recente (pelo formato de data no nome do arquivo)
contratos_files = list(raw_data_dir.glob('contratos_amostra_*.csv'))

if contratos_files:
    # Pega o arquivo mais recente baseado na data de modificação
    latest_contratos_file = max(contratos_files, key=lambda x: x.stat().st_mtime)

    # Verificação (para debug)
    print(f'Arquivo de contratos encontrado: {latest_contratos_file}')
    print(f'O arquivo existe? {latest_contratos_file.exists()}')

    # Carrega o Dataframe
    df = pd.read_csv(latest_contratos_file, encoding='utf-8', low_memory=False)

    # Visualiza dimensões e primeiras linhas
    print(f'\nDataframe carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')
    print(f'Primeira 5 colunas: {', '.join(df.columns[:5])}')

else:
    print('ERRO: Nenhum arquivo de contratos encontrado em data/raw')
    print(f'Diretório verificado: {raw_data_dir.resolver()}')
    print(f'O diretório existe? {raw_data_dir.exists()}')
    print('Arquivos no diretório:', list(raw_data_dir.glob('*')))

Arquivo de contratos encontrado: ..\data\raw\contratos_amostra_2025-08-03.csv
O arquivo existe? True

Dataframe carregado: 12000 linhas, 39 colunas
Primeira 5 colunas: codigoOrgao, nomeOrgao, codigoUnidadeGestora, nomeUnidadeGestora, codigoUnidadeGestoraOrigemContrato


In [ ]:
df.head(10)

Verificação de Nulos por quantidade e porcentagem 👇

In [2]:
# Calcula contagem e percentual de nulos em cada coluna
nulos = df.isnull().sum()
percentual = (df.isnull().sum() / len(df) * 100).round(2)

# Cria um Dataframe simples apenas com colunas que têm pelo menos um valor nulo
colunas_com_nulos = pd.DataFrame({
    'Valores Nulos': nulos[nulos >0],
    'Percentual (%)': percentual[nulos > 0]
}).sort_values('Percentual (%)', ascending=False)

# Exibe o resultado
colunas_com_nulos

,Valores Nulos,Percentual (%)
numeroControlePncpContrato,12000,100.00
dataHoraExclusao,12000,100.00
totalDespesasAcessorias,11999,99.99
nomeSubcategoria,11462,95.52
codigoSubcategoria,11462,95.52
unidadesRequisitantes,10824,90.20
informacoesComplementares,8196,68.30
valorAcumulado,6287,52.39
codigoTipo,4997,41.64
nomeTipo,4997,41.64


- `nomeRazaoSocialFornecedor` -> dá para descobrir os nulos baseado no 'niFornecedor'
- `valorAcumulado` -> dá para descobrir alguns nulos baseado nas colunas 'valorGlobal', 'numeroParcelas' e 'valorParcela' (manter cuidado ao fazer isso para evitar viés de dados -> olhar melhor depois)

OBS sobre algumas colunas que não irei excluir ainda:
- `informacoesComplementares` -> pode ser útil para descobrir alguns nulos faltantes (ou valores errados) de outras colunas
- `codigoTipo` e `nomeTipo´ -> analisar se convém manter ou excluir (levando em conta que mais de 90% dos registros não nulos são empenho)

Exclusão de algumas colunas 👇

In [ ]:
colunas_para_excluir = [
    'numeroControlePncpContrato', # muitos nulos
    'dataHoraExclusao', # muitos nulos
    'totalDespesasAcessorias', # muitos nulos
    'nomeSubcategoria', # muitos nulos
    'codigoSubcategoria', # muitos nulos
    'unidadesRequisitantes', # muitos nulos // repensar se necessário
    




    'contratoExcluido', # não é uma coluna de interesse
    ]

df.drop(colunas_para_excluir, axis=1, inplace=True)

In [6]:
len(df.columns)

39